In [2]:
from bs4 import BeautifulSoup

import pandas as pd

import asyncio
import aiohttp
import time

In [1]:
def temizle(metin):
  metin = metin.replace("\n", " ").replace("\t", " ").replace("\r", " ")
  while "  " in metin:
    metin = metin.replace("  ", " ")
  return metin.strip()

In [3]:
tum_sorular = []
async def sayfadaki_sorulari_cek(session, i):
  try:
    async with session.get('https://www.hukuksorucevap.com.tr/sorucevap/?sayfa=' + str(i)) as response:
      response.encoding = 'utf-8'
      soup = BeautifulSoup(await response.text(), 'html.parser')
      soru = soup.find('div', {'id': 'Soru'})
      sorular = soru.find_all('li')
      for soru_li in sorular:
          soru_dict = {}
          soru_metni = soru_li.find('div', {'id': 'Soru'}).text
          soru_dict['soru'] = temizle(soru_metni)

          cevap_metni = soru_li.find('div', {'id': 'Cevap'}).find('div', {'class': 'DoktorCevabi'}).text
          soru_dict['cevap'] = temizle(cevap_metni)

          cevaplayan_unvan = soru_li.find('div', {'id': 'Cevap'}).find('div', {'class': 'DoktorUnvani'}).text
          soru_dict['cevaplayan_unvan'] = temizle(cevaplayan_unvan)

          tum_sorular.append(soru_dict)
  except Exception as e:
    print(e)

In [4]:
start_time = time.time()
async with aiohttp.ClientSession() as session:
    tasks = []
    for i in range(1, 37):        
        tasks.append(sayfadaki_sorulari_cek(session, i))
    await asyncio.gather(*tasks)
end_time = time.time()

print(f"Time taken to get the details of profiles: {end_time - start_time} seconds")

Time taken to get the details of profiles: 0.622565746307373 seconds


In [5]:
# convert tum_sorular to a DataFrame
df = pd.DataFrame(tum_sorular)
df

,soru,cevap,cevaplayan_unvan
0,"Merhabalar, Bir kamu kurumunda sözleşmeli işçi...","Merhaba, söz konusu sizin durumunuza ilişkin o...",Hukuk Bilgi Platformu
1,Küçük yaşta mahkeme kararı ile evlatlık verild...,"Merhaba, evet siz her iki tarafında mirasçısıs...",Hukuk Bilgi Platformu
2,"Merhaba, biz evimizi kiraya verdik ve başka bi...","Merhaba, söz konusu sizin durumunza ilişkin ol...",Hukuk Bilgi Platformu
3,Merhaba nişanlım 2021 karar nolu cezasından de...,"Merhaba, denetimli serbestlik uygulamasında an...",Hukuk Bilgi Platformu
4,Merhaba 01/09/1999 tarihinde bir dershane ile ...,"Merhaba, bu konuya ilişkin olarak hizmet tespi...",Hukuk Bilgi Platformu
...,...,...,...
349,İş görüşmesi amacıyla is evrakları arasında se...,"Merhaba, mağduriyetinizi içeren bir dilekçe ha...",Hukuk Bilgi Platformu
350,Merhaba abim babamdan beş yıl önce vefat etti ...,"Merhaba, mirasın kabulü ile terekedeki aktif v...",Hukuk Bilgi Platformu
351,Ev sahibimizin kız kardeşi kardeşi evlenicekmi...,"İyi Günler, ihtiyaç sebebiyle tahliye davası a...",Hukuk Bilgi Platformu
352,"Merhaba, ben 3 yıl önce arkadaşımla bir ev kir...","Merhaba, sizi evden çıkarabilmeleri için sizin...",Hukuk Bilgi Platformu


In [6]:
df.to_csv('hukuksorucevap.csv', index=False, encoding='utf-8')

In [8]:
df_csv = pd.read_csv('hukuksorucevap.csv')
df_csv

,soru,cevap,cevaplayan_unvan
0,"Merhabalar, Bir kamu kurumunda sözleşmeli işçi...","Merhaba, söz konusu sizin durumunuza ilişkin o...",Hukuk Bilgi Platformu
1,Küçük yaşta mahkeme kararı ile evlatlık verild...,"Merhaba, evet siz her iki tarafında mirasçısıs...",Hukuk Bilgi Platformu
2,"Merhaba, biz evimizi kiraya verdik ve başka bi...","Merhaba, söz konusu sizin durumunza ilişkin ol...",Hukuk Bilgi Platformu
3,Merhaba nişanlım 2021 karar nolu cezasından de...,"Merhaba, denetimli serbestlik uygulamasında an...",Hukuk Bilgi Platformu
4,Merhaba 01/09/1999 tarihinde bir dershane ile ...,"Merhaba, bu konuya ilişkin olarak hizmet tespi...",Hukuk Bilgi Platformu
...,...,...,...
349,İş görüşmesi amacıyla is evrakları arasında se...,"Merhaba, mağduriyetinizi içeren bir dilekçe ha...",Hukuk Bilgi Platformu
350,Merhaba abim babamdan beş yıl önce vefat etti ...,"Merhaba, mirasın kabulü ile terekedeki aktif v...",Hukuk Bilgi Platformu
351,Ev sahibimizin kız kardeşi kardeşi evlenicekmi...,"İyi Günler, ihtiyaç sebebiyle tahliye davası a...",Hukuk Bilgi Platformu
352,"Merhaba, ben 3 yıl önce arkadaşımla bir ev kir...","Merhaba, sizi evden çıkarabilmeleri için sizin...",Hukuk Bilgi Platformu


In [ ]:
# https://www.hukuksorucevap.com.tr/sorucevap/?sayfa=1 adresindeki soru ve cevapları çekeceğiz. Toplamda 36 sayfa var.

